In [1]:
import numpy as np
from hmmlearn import hmm

In [14]:
class PairwiseHMM(hmm.CategoricalHMM):
    def __init__ (self, n_features, delta, tau, epsilon):

        #Los cinco estados son (Begin, M, X, Y, End)
        super().__init__(n_components=5, n_features= n_features)

        self.startprob_ = np.array([1,0,0,0,0])

        self.transmat_  = np.array([[0,1-2*delta-tau, delta, delta, tau], 
                                   [0,1-2*delta-tau, delta, delta, tau],
                                   [0,1-epsilon-tau, epsilon, 0, tau],
                                   [0,1-epsilon-tau, 0, epsilon, tau],
                                   [0,0,0,0,1]]) 


    '''def _check(self):
        #Llamamos a check del abuelo
        super(hmm.CategoricalHMM, self)._check()
        
        #Elimino la restricción de que la suma de filas de la matriz de emisión tenga que ser 1
        self.emissionprob_ = np.atleast_2d(self.emissionprob_)
        if self.n_features is None:
            self.n_features = self.emissionprob_.shape[1]
        if self.emissionprob_.shape != (self.n_components, self.n_features):
            raise ValueError(
                f"emissionprob_ must have shape"
                f"({self.n_components}, {self.n_features})")'''
        
    def modifiedViterbi(self, x, y):
        #Tendremos valores de (0,0) a (n,m)
        n = len(x)
        m = len(y)

        v_M = np.zeros((n+1, m+1))
        v_X = np.zeros((n+1, m+1))
        v_Y = np.zeros((n+1, m+1))

        pointer_M = [[None*(m+1)]*(n+1)]
        pointer_X = [[None*(m+1)]*(n+1)]
        pointer_Y = [[None*(m+1)]*(n+1)]

        v_M[0][0] = 1

        for i in range(1,n+1):
            for j in range(1,m+1):
                

In [9]:
model = PairwiseHMM(25,1/10,1/20,1/10)

In [12]:
'''En este caso, las emisiones en cada fila son de forma:
(A:A, A:C, A:G, A:T, A:_, C:A, C:C, C:G, C:T, C:_, G:A, G:C, G:G, G:T, G:_, T:A, T:C, T:G, T:T, T:_, _:A, _:C, _:G, _:T, _:_ )
'''
model.emissionprob_=np.array(
[ [ 1/5 if (i % 6==0)  else 1/60 if (i % 5==4) else 0 for i in range(25)],
  [ 1/5 if (i % 6==0)  else 1/60 if (i % 5==4) else 0 for i in range(25)],
  [ 1/4 if (i >=20 and i<24) else 0 for i in range(25)],
  [ 1/4 if (i % 5==4 and i<24) else 0 for i in range(25)],
  [ 1 if (i==24) else 0 for i in range(25)]
]
)

In [18]:
def decodify(x):
    return np.array([0 if letter == 'A' else 1 if letter=='C' else 2 if letter=='G' else 3 for letter in x.upper()])

In [19]:
decodify("agcgtta")

array([0, 2, 1, 2, 3, 3, 0])